In [ ]:
import torch
import torch.nn as nn
from os.path import isfile

from config import *
from utils import BaselineTrainer, GNNTrainer
from gnn import GNNModel
from utils import TrafficVolumeDataLoader, TrafficVolumeGraphDataLoader, create_edge_index_and_features

%run -i 'preprocess_data.py'

In [ ]:
class BaseLineModel(nn.Module):
    def __init__(self):
        super().__init__()
        input_size = 98
        output_size = 95
        self.fcnn = nn.Sequential(
            # TODO Implement the feedforward neural network here
        )

    def forward(self, x):
        x = self.fcnn(x)
        return x

In [12]:
config = configs[0]
config["checkpoint_file"] = "checkpoints/NN.pth"
config["epochs"] = 5
config_baseline["val_per_epoch"] = 1
name = config["name"]
lr = config["lr"]
batch_size = config["batch_size"]
loss_function = nn.L1Loss()

device = 'cpu'
model = BaseLineModel()
train_dataloader = TrafficVolumeDataLoader(train_data_file, batch_size, num_workers, shuffle=True)
val_dataloader = TrafficVolumeDataLoader(val_data_file, batch_size, num_workers)
test_dataloader = TrafficVolumeDataLoader(test_data_file, batch_size, num_workers)
trainer = BaselineTrainer(model, train_dataloader, val_dataloader, test_dataloader, config, loss_function, device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr) 
trainer.train(optimizer)

# Evaluate model on test data and compute test loss
trainer.evaluate()

# Make some prediction and save plot
from_index = 500 
length = 500
trainer.save_prediction_plot(from_index, length)

In [ ]:
config = configs[1]
name = config["name"]
lr = config["lr"]
batch_size = config["batch_size"]
loss_function = nn.L1Loss()

device = 'cpu'
model = GNNModel()
edge_index, edge_weight = create_edge_index_and_features(stations_included_file, stations_data_file, graph_file)
train_dataloader = TrafficVolumeGraphDataLoader(train_data_file, edge_index, edge_weight, batch_size, num_workers, shuffle=True)
val_dataloader = TrafficVolumeGraphDataLoader(val_data_file, edge_index, edge_weight, batch_size, num_workers)
test_dataloader = TrafficVolumeGraphDataLoader(test_data_file, edge_index, edge_weight, batch_size, num_workers)
trainer = GNNTrainer(model, train_dataloader, val_dataloader, test_dataloader, config, loss_function, device)
    
trainer.print_model_size()

# Evaluate model on test data and compute test loss
trainer.evaluate()

# Make some prediction and save plot
from_index = 500 
length = 500
trainer.save_prediction_plot(from_index, length)